## Option pricing

Text.

---

---

### Black-Scholes model

Given a stock $S(t)$ is modelled under the risk-neutral Geometric Brownian Motion (GBM) model, the Black-Scholes model allows to compute the theoretical value of an option contract for European options based on five input variables:

![](./img/bs-model-overview.png)

for which the dynamics of the stock and option value, under risk-neutral measure $\mathbb{Q}$, are given by

\begin{align}
	dS(t) &= \mu \cdot S(t) \cdot dt + \sigma \cdot S(t) \cdot dW^{\mathbb{Q}}(t),
	\\
	dV(t, S) &= \mu \cdot S(t) \cdot dt + \sigma \cdot S(t) \cdot dW(t),
\end{align}

which yields the Black-Scholes pricing PDE

\begin{equation}
	d \mathbf{X}
	= \bar{\mathbf{\mu}}dt + \mathbf{D}\mathbf{L} d\tilde{\mathbf{W}}
	= \bar{\mathbf{\mu}}dt + \bar{\mathbf{\sigma}} d\tilde{\mathbf{W}},
\end{equation}

with a terminal condition $V(T, S) = H(T, S)$ specified by some payoff function $H(T, S)$ at time $T$, based on whether the type of the option contract (CALL or PUT).

The **Feynman-Kac theorem** allows to express the solution to a PDE with a terminal condition (in this case the Black-Scholes pricing PDE) in terms of an expectation

\begin{equation}
	V(t) = e^{-r(T-t)} \cdot \mathbb{E}^{\mathbb{Q}} [H(T,S) | \mathcal{F}(t)],
\end{equation}

where the expectation can be solved using simulation or integration. An analytical solution then given by

\begin{equation}
	V(t) = e^{-r(T-t)} \cdot \mathbb{E}^{\mathbb{Q}} [H(T,S) | \mathcal{F}(t)].
\end{equation}